In [1]:
import csv
import re
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
## file name and setup 

file_name = 'index'

#pollutants
terms_of_interest = ['carcinogenic', 'carcinogens', 'arsenic', 'nitrate', 'nitrite', 'nitrates']

#urls
basic_url = 'https://www.communitywatercenter.org/news?n=448&page='
n_pages = 34

#cities
location_names = pd.read_csv('CA_names.csv')

In [3]:
pagelinks =[]

for number in range(1,n_pages+1):

    url = basic_url + str(number)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser') #containing the HTML of the page
    name_box = soup.find_all('h3', attrs={'class': 'news_heading'})
    for count in range(0, len(name_box)):
            for link in name_box[count].find_all('a', href=True):
                pagelinks.append(link['href']) 

In [4]:
# For finding cities and counties

cities = location_names['city'].tolist() 
#counties = location_names['county_name'].tolist()
counties = ['Tulare', 'Kern']

## hack solution to control for weird city names matching words

cities.remove('August')
cities.remove('Challenge')
#locations = cities + counties

#creating regex expressions
re_cities = '' + cities[0]
for i in cities[1:]:
    re_cities = re_cities + "|" + i
re_counties = '' + counties[0]
for i in counties[1:]:
    re_counties = re_counties + "|" + i
    
def intersect(a, b):
     return list(set(a) & set(b))
    
def most_common(lst):
    return max(set(lst), key=lst.count)

In [11]:
# Writing to CSV file (basic)

data = []

for count in range(0, len(pagelinks)):
    
    page_url = "https://www.communitywatercenter.org" + pagelinks[count]
    page = urlopen(page_url)
    soup = BeautifulSoup(page, 'html.parser')
    main = soup.find("div", id="content").text.strip()
    
    title = soup.title.string.replace('\r', '').replace('\n', '')
    date_section = soup.find('div', id="headline")
    date = date_section.find_all(string=re.compile("Published on"))
    
    list_city = re.findall(re_cities, main)
    list_county = re.findall(re_counties, main)
    if len(list_city) > 0:
        main_city = most_common(list_city)
    else:
        main_city = ""
    if len(list_county) > 0:
        main_county = most_common(list_county)
    else:
        main_county = ""
    
    city = list(set(list_city))
    county = list(set(list_county))
    
    for term in range(0, len(terms_of_interest)):

            phrase = [(sentence + '.').replace('\r', '').replace('\n', '') for sentence in main.split('.') if terms_of_interest[term] in sentence]

            for item in range(0, len(phrase)):

                data.append([phrase[item], terms_of_interest[term], str(date)[19:-8], city, county, main_city, main_county, title, page_url])
                
phrase_df = pd.DataFrame(data, columns = ["phrase", "contaminant", "date", "city", "county","main city","main county", "title", "page_url"])
               
#     with open(file_name + '.csv', 'a') as mycsv: ## a for append
        
#         writer = csv.writer(mycsv)
        
#         for term in range(0, len(terms_of_interest)):

#             phrase = [sentence + '.' for sentence in main.split('.') if terms_of_interest[term] in sentence]

#             for item in range(0, len(phrase)):

#                 writer.writerow([phrase[item], str(date)[19:-8], city, county, title, page_url])

,phrase,contaminant,date,city,county,main city,main county,title,page_url
0,"NEW OFFICE IN WATSONVILLE, CAWe are excited t...",nitrate,"September 17, 2018","[Watsonville, Fresno, San Joaquin, Porterville...",[],Visalia,,August 2018 eNews - Community Water Center,https://www.communitywatercenter.org/august201...
1,"Bartolo Chavez of Arvin, CA has drinking water...",carcinogenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...
2,"Bartolo Chavez of Arvin, CA has drinking water...",arsenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...
3,"org #NoToxicTaps1,2,3 - TCP1,2,3-trichloroprop...",carcinogenic,"July 31, 2018","[Orosi, Tulare, Fresno, San Joaquin, Portervil...","[Kern, Tulare]",Porterville,Tulare,July 2018 eNews - Community Water Center,https://www.communitywatercenter.org/july_2018...
4,Over 300 mostly rural and economically disadv...,arsenic,"October 24, 2017",[],[],,,Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...


In [19]:
ids = phrase_df["phrase"]
phrase_df[ids.isin(ids[ids.duplicated()])]#.sort("phrase")

,phrase,contaminant,date,city,county,main city,main county,title,page_url,new_contaminant
1,"Bartolo Chavez of Arvin, CA has drinking water...",carcinogenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...,carcinogenic
2,"Bartolo Chavez of Arvin, CA has drinking water...",arsenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...,arsenic
4,Over 300 mostly rural and economically disadv...,arsenic,"October 24, 2017",[],[],,,Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...,arsenic
5,Over 300 mostly rural and economically disadv...,nitrate,"October 24, 2017",[],[],,,Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...,nitrate
6,Over 300 mostly rural and economically disadv...,nitrates,"October 24, 2017",[],[],,,Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...,nitrates
9,Waiting until the next legislative session wo...,arsenic,"September 09, 2017",[],[],,,"FARMERS, ENVIRONMENTAL LEADERS URGE LEGISLATUR...",https://www.communitywatercenter.org/kvpr_sb623,arsenic
10,Waiting until the next legislative session wo...,nitrate,"September 09, 2017",[],[],,,"FARMERS, ENVIRONMENTAL LEADERS URGE LEGISLATUR...",https://www.communitywatercenter.org/kvpr_sb623,nitrate
11,Waiting until the next legislative session wo...,nitrates,"September 09, 2017",[],[],,,"FARMERS, ENVIRONMENTAL LEADERS URGE LEGISLATUR...",https://www.communitywatercenter.org/kvpr_sb623,nitrates
12,Some area's water containing over three times...,carcinogens,"August 11, 2017",[],[],,,T is for Toxic: Danger Lurking in California S...,https://www.communitywatercenter.org/toxic_sch...,carcinogens
13,Some area's water containing over three times...,arsenic,"August 11, 2017",[],[],,,T is for Toxic: Danger Lurking in California S...,https://www.communitywatercenter.org/toxic_sch...,arsenic


In [24]:
phrase_df.head()

,phrase,contaminant,date,city,county,main city,main county,title,page_url,new_contaminant
0,"NEW OFFICE IN WATSONVILLE, CAWe are excited t...",nitrate,"September 17, 2018","[Watsonville, Fresno, San Joaquin, Porterville...",[],Visalia,,August 2018 eNews - Community Water Center,https://www.communitywatercenter.org/august201...,nitrate
1,"Bartolo Chavez of Arvin, CA has drinking water...",carcinogenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...,carcinogenic
2,"Bartolo Chavez of Arvin, CA has drinking water...",arsenic,"September 06, 2018",[Arvin],[],Arvin,,Justice Delayed - Community Water Center,https://www.communitywatercenter.org/justicede...,arsenic
3,"org #NoToxicTaps1,2,3 - TCP1,2,3-trichloroprop...",carcinogenic,"July 31, 2018","[Orosi, Tulare, Fresno, San Joaquin, Portervil...","[Kern, Tulare]",Porterville,Tulare,July 2018 eNews - Community Water Center,https://www.communitywatercenter.org/july_2018...,carcinogenic
4,Over 300 mostly rural and economically disadv...,arsenic,"October 24, 2017",[],[],,,Small Communities Can't Solve Toxic Water Prob...,https://www.communitywatercenter.org/toxic_wat...,arsenic


In [25]:
phrase_df.to_csv("phrase.csv")

In [26]:
#read csv
output = pd.read_csv('phrase.csv')
# isolate phrases without location

In [27]:
## for data analysis %load_ext rpy2.ipython 